In [ ]:
#Importing required modules
import numpy as np
import pandas as pd
import pickle
import time
import scipy
import sklearn

import feedparser as fp
import html
from nltk.tokenize import TreebankWordTokenizer,WhitespaceTokenizer,word_tokenize
from gensim.summarization import keywords
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from langdetect import detect
from website.code_bin import Cleaner
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from multiprocessing import Pool
import string
from newspaper import Article

## Models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer


# Plotting
import matplotlib.pyplot as plt

# Word2Vec

In [ ]:
'''load pretrained word2vec (on Google news dataset)'''
import gensim
import numpy as np

floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/data/'
fname = 'GoogleNews-vectors-negative300.bin'

word2vec = gensim.models.KeyedVectors.load_word2vec_format(floc+fname, binary=True)

In [ ]:
'''define functions for parsing tokenized episode summaries'''
def ep2vec(episode):
    n = 0
    a = []
    for w in episode:
        try:
            v = word2vec[w]
            if((np.isnan(v).any() + np.isinf(v).any()) == 0):
                a.append(v)
                n += 1
        except KeyError as e:
            pass
    if(n==0):
        return np.full([300,], np.nan)
    avg = np.mean(np.array(a),axis=0)
    return avg
    
def pod2vec(episodes):
    return np.nanmean(np.asarray([ep2vec(e) for e in episodes]),axis=0)

In [ ]:
#load in preprocessed text summaries and word2vec them


floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries6/'
fctr = 0
associator = '1580538675'
nfile = 7
vectorized_eps = []
for i in range(0,nfile):
    start_time = time.time()
    fname = 'preprocessed_summaries' + str(fctr) +'_'+ associator + '.pkl'
    with open(floc+fname,'rb') as fid:
         pps = pickle.load(fid)
    for p in pps:
        vectorized_eps.append((p[0],pod2vec(p[1]),[ep2vec(e) for e in p[1]]))
    fctr += 1
    stop_time = time.time()
    duration = stop_time - start_time
    print('Done with file ' + str(i) + ' (duration=' + str(duration) + ')')
print('done!')


#save vector representations

floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries9/'

associator = str(int(np.floor(time.time())))
fname = 'vectorized_summaries'+'_'+ associator + '.pkl'
with open(floc+fname,'wb') as fid:
    pickle.dump(vectorized_eps,fid)

print('done!')

# GloVe

In [ ]:
'''load pretrained glove vector (on Wikipedia)'''
import gensim
import numpy as np

floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/data/'
fname='glove.6B.300d.txt'

Glove = {}
with open(floc+fname, 'r', encoding='utf8',errors = 'ignore') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32", )
        Glove[word] = vector
floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/Data/'
fname='glove_vecs_6B.pkl'
with open(floc+fname,'wb') as fid:
         pickle.dump(Glove,fid)  
 

In [ ]:
'''Save the pre-trained Glove vectors into a pickle file'''

floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/data/'
fname='glove_vecs_6B.pkl'
with open(floc+fname,'rb') as fid:
    Glove=pickle.load(fid)  

In [ ]:
'''define functions for finding GloVe Vectors of episode summaries'''
def ep2glove(episode):
    n = 0
    a = []
    for w in episode:
        try:
            v = Glove[w]
            if((np.isnan(v).any() + np.isinf(v).any()) == 0):
                a.append(v)
                n += 1
        except KeyError as e:
            pass
    if(n==0):
        return np.full([300,], np.nan)
    avg = np.mean(np.array(a),axis=0)
    return avg
    
def pod2glove(episodes):
    return np.nanmean(np.asarray([ep2vec(e) for e in episodes]),axis=0)

In [ ]:
#load in preprocessed text summaries and glove them


floc = '/Users/mohsenghassemi/Desktop/Job_Material/Insight/Project/summaries6/'
fctr = 0
associator = '1580326866'
nfile = 4
vectorized_eps = []
for i in range(1,nfile):
    start_time = time.time()
    fname = 'preprocessed_summaries' + str(fctr) +'_'+ associator + '.pkl'
    with open(floc+fname,'rb') as fid:
         pps = pickle.load(fid)
    for p in pps:
        vectorized_eps.append((p[0],pod2glove(p[1]),[ep2glove(e) for e in p[1]]))
    fctr += 1
    stop_time = time.time()
    duration = stop_time - start_time
    print('Done with file ' + str(i) + ' (duration=' + str(duration) + ')')
print('done!')


#save vector representations

associator = str(int(np.floor(time.time())))
fname = 'glove_vectors_all'+'_'+ associator + '.pkl'
with open(floc+fname,'wb') as fid:
    pickle.dump(vectorized_eps,fid)

print('done!')

# Sentence BERT

In [ ]:
'''Load the pre-trained model on Wikipedia 2014 and Gigaword 5'''
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
'''Load preprocessed text data'''
with open('processed_BERT_first4sentences','rb') as fid:
         sentence_df=pickle.load(fid)

In [ ]:
sentences_df['BERT_vecs']=sentences_df['episodes_descriptions_sentences'].apply(lambda x: model_BERT.encode(x))
with open('processed_BERT_sentences_vecs','wb') as fid:
         pickle.dump(sentences_df,fid)